In [3]:
from raygun.modelv3.raygun import Raygun
from raygun.modelv3.ltraygun import RaygunLightning

In [12]:
checkpoint = "/hpc/group/singhlab/rawdata/raygunv2models/species_function_aware_sep30_val_blosum_0.9856.ckpt"

In [5]:
from esm.pretrained import esm2_t33_650M_UR50D

emodel, ealph = esm2_t33_650M_UR50D()

In [6]:
raymodel = Raygun(numdecoders=12, 
                 numencoders=12, 
                 fixed_esm_batching=True)

In [7]:
import inspect
print(inspect.getsource(RaygunLightning.__init__))

    def __init__(self, raygun, esmmodel,
                lr = 1e-3, 
                crossentropyloss = 1., 
                reconstructionloss = 1., 
                replicateloss = 1.,
                log_wandb = False,
                traininglog = "traininglog.txt",
                finetune = False):
        super().__init__()
        self.model            = raygun
        self.esmmodel         = esmmodel
        self.lr               = lr
        self.crossentropyloss = crossentropyloss
        self.reconstructloss  = reconstructionloss
        self.replicateloss    = replicateloss
        self.trainlosses      = defaultdict(list)
        self.vallosses        = defaultdict(list)
        self.epoch            = 0
        bl                    = substitution_matrices.load("BLOSUM62")
        self.blosummat        = pd.DataFrame(bl, columns = list(bl.alphabet))
        self.blosummat.index  = list(bl.alphabet)
        self.decodermodel     = raygun.esmdecoder
        
        self.e

In [13]:
ltraygun = RaygunLightning.load_from_checkpoint(checkpoint,
                                                raygun=raymodel,
                                               esmmodel=emodel,
                                               strict=False)

/hpc/group/singhlab/user/rs670/micromamba/envs/raygun/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:191: Found keys that are in the model state dict but not in the checkpoint: ['esmmodel.embed_tokens.weight', 'esmmodel.layers.0.self_attn.k_proj.weight', 'esmmodel.layers.0.self_attn.k_proj.bias', 'esmmodel.layers.0.self_attn.v_proj.weight', 'esmmodel.layers.0.self_attn.v_proj.bias', 'esmmodel.layers.0.self_attn.q_proj.weight', 'esmmodel.layers.0.self_attn.q_proj.bias', 'esmmodel.layers.0.self_attn.out_proj.weight', 'esmmodel.layers.0.self_attn.out_proj.bias', 'esmmodel.layers.0.self_attn.rot_emb.inv_freq', 'esmmodel.layers.0.self_attn_layer_norm.weight', 'esmmodel.layers.0.self_attn_layer_norm.bias', 'esmmodel.layers.0.fc1.weight', 'esmmodel.layers.0.fc1.bias', 'esmmodel.layers.0.fc2.weight', 'esmmodel.layers.0.fc2.bias', 'esmmodel.layers.0.final_layer_norm.weight', 'esmmodel.layers.0.final_layer_norm.bias', 'esmmodel.layers.1.self_attn.k_proj.weight', 'esmmodel.layers.1

In [14]:
from io import StringIO
from Bio import SeqIO

gfprec = """>sp|P42212|GFP_AEQVI Green fluorescent protein OS=Aequorea victoria OX=6100 GN=GFP PE=1 SV=1
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTL
VTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLV
NRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLAD
HYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
"""
gfpseq = str(SeqIO.read(StringIO(gfprec), "fasta").seq)
len(gfpseq)

238

In [17]:
_, _, tok = ealph.get_batch_converter()([("gfp", gfpseq)])
emodel    = emodel.eval().to(0)
embed     = emodel.forward(tok.to(0), return_contacts=False, repr_layers=[33])["representations"][33][:, 1:-1, :]

In [19]:
ltraymodel = ltraygun.model.eval().to(0)

In [20]:
payload = ltraymodel.forward(embed, return_logits_and_seqs=True)

RuntimeError: Tensor type unknown to einops <class 'tuple'>